In [1]:
import json
import time
import pandas as pd
import requests
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
import quandl
from config import api_key
import csv
import os

# Master Data - CSV

In [55]:
master_data_path = os.path.join("..", "..", "data", "project_one_data", "master_data_cleaned.csv")
master_data_df = pd.read_csv(master_data_path)
master_data_df.head()

,Year,Month,Year-Month,Date Zillow,Date Interest,Indicator ID,Indicator Name,Region ID,Region Type,Region Name,Value,County Population,Unemployment Rate,Wells Fargo Rate,Freddie Mac Rate
0,2016,Apr,2016-4,4/30/2016,4/28/2016,Z2BR,ZHVI 2-Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",384884.0,30566.0,3.5,3.88,3.66
1,2016,Apr,2016-4,4/30/2016,4/28/2016,Z4BR,ZHVI 4-Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",881951.0,30566.0,3.5,3.88,3.66
2,2016,Apr,2016-4,4/30/2016,4/28/2016,Z5BR,ZHVI 5+ Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",1287769.0,30566.0,3.5,3.88,3.66
3,2016,Apr,2016-4,4/30/2016,4/28/2016,ZATT,ZHVI All Homes- Top Tier Time Series ($),753871,metro,"Breckenridge, CO; CO",842653.0,30566.0,3.5,3.88,3.66
4,2016,Apr,2016-4,4/30/2016,4/28/2016,Z1BR,ZHVI 1-Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",243688.0,30566.0,3.5,3.88,3.66


# SSAM - Median Prices, All Homes

In [52]:
# Look at SSAM - All Homes Monthly
ssam_df = master_data_df[master_data_df["Indicator ID"] == "SSAM"]

# SSSM - Median Prices, Single Family Residence

This section looks at all single family residence median prices over time from April 2016 to August 2020. 

## Master SSSM DataFrame

In [59]:
# Look at SSSM - SFR Only
sssm_df = master_data_df[master_data_df["Indicator ID"] == "SSSM"]
sssm_df.head()

,Year,Month,Year-Month,Date Zillow,Date Interest,Indicator ID,Indicator Name,Region ID,Region Type,Region Name,Value,County Population,Unemployment Rate,Wells Fargo Rate,Freddie Mac Rate
411,2016,Apr,2016-4,4/30/2016,4/28/2016,SSSM,"Median Sale Price (Smooth, SFR only, Monthly)",394484,metro,"Colorado Springs, CO",249350.0,689481.0,3.5,3.88,3.66
558,2016,May,2016-5,5/31/2016,5/5/2016,SSSM,"Median Sale Price (Smooth, SFR only, Monthly)",394484,metro,"Colorado Springs, CO",254975.0,689481.0,3.4,3.75,3.61
559,2016,May,2016-5,5/31/2016,5/12/2016,SSSM,"Median Sale Price (Smooth, SFR only, Monthly)",394484,metro,"Colorado Springs, CO",254975.0,689481.0,3.4,3.75,3.57
560,2016,May,2016-5,5/31/2016,5/19/2016,SSSM,"Median Sale Price (Smooth, SFR only, Monthly)",394484,metro,"Colorado Springs, CO",254975.0,689481.0,3.4,3.88,3.58
561,2016,May,2016-5,5/31/2016,5/26/2016,SSSM,"Median Sale Price (Smooth, SFR only, Monthly)",394484,metro,"Colorado Springs, CO",254975.0,689481.0,3.4,3.88,3.64


In [43]:
sssm_all = master_data_df[(master_data_df["Indicator ID"] =="SSSM")]
sssm_all = sssm_all.sort_values("Year-Month")
sssm_all = sssm_all.groupby("Year-Month").first()

In [44]:
all_sssm_unemp_x = sssm_all["Unemployment Rate"]
all_sssm_value_y = sssm_all["Value"]

correlation = st.pearsonr(all_sssm_unemp_x,all_sssm_value_y)
print(f"The correlation between the unemployment rate and median sale price is: {correlation[0]}.")
print(f"P-Value is: {correlation[1]}.")

The correlation between the unemployment rate and median sale price is: 0.3214883502129601.
P-Value is: 0.018907369025041733.


In [45]:
all_sssm_unemp_x = sssm_all["Unemployment Rate"]
all_sssm_fmr_y = sssm_all["Freddie Mac Rate"]

correlation = st.pearsonr(all_sssm_unemp_x,all_sssm_fmr_y)
print(f"The correlation between the unemployment rate and median sale price is: {correlation[0]}.")
print(f"P-Value is: {correlation[1]}.")

The correlation between the unemployment rate and median sale price is: -0.4809874842567612.
P-Value is: 0.00026668296365345615.


### Correlation: All Unemployed, Median Sale Price

In [42]:
all_sssm_unemp_x = sssm_df["Unemployment Rate"]
all_sssm_value_y = sssm_df["Value"]

correlation = st.pearsonr(all_sssm_unemp_x,all_sssm_value_y)
print(f"The correlation between the unemployment rate and median sale price is: {correlation[0]}.")
print(f"P-Value is: {correlation[1]}.")

The correlation between the unemployment rate and median sale price is: 0.24332148502681336.
P-Value is: 1.6353361929322935e-10.


### Correlation: Greeley - Unemployed, Median Sale Price

In [ ]:
sssm_df_greely = sssm_df[sssm_df["Region Name"].str.contains("Greeley")]

In [ ]:
greeley_sssm_unemp_x = sssm_df_greely["Unemployment Rate"]
greeley_sssm_value_y = sssm_df_greely["Value"]

correlation = st.pearsonr(greeley_sssm_unemp_x,greeley_sssm_value_y)
print(f"The correlation between the unemployment rate and median sale price is: {correlation[0]}.")
print(f"P-Value is: {correlation[1]}.")

In [ ]:
# Formula
(slope, intercept, rvalue, pvalue, stderr) = linregress(greeley_sssm_unemp_x, greeley_sssm_value_y)
greeley_regress_values = greeley_sssm_unemp_x * slope + intercept

# Line Equation
greeley_line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot
plt.scatter(greeley_sssm_unemp_x,greeley_sssm_value_y)
plt.plot(greeley_sssm_unemp_x,greeley_regress_values,"r-")

# Design Plot
plt.annotate(greeley_line_eq,(6, 280000),fontsize=15,color="red")
plt.title("Greeley, CO \n Unemploment % vs. Median Sale Price")
plt.xlabel("Unemployment %")
plt.ylabel('Median Sale Price')

# Show
plt.show()

### Correlation: Colorado Springs - Unemployed, Median Sale Price

In [40]:
sssm_df_cosprings = master_data_df[(master_data_df["Indicator ID"] =="SSSM") & master_data_df["Region Name"].str.contains("Colorado Springs")]
sssm_df_cosprings = sssm_df_cosprings.groupby("Year-Month").first()

In [41]:
cosprings_sssm_unemp_x = sssm_df_cosprings["Unemployment Rate"]
cosprings_sssm_value_y = sssm_df_cosprings["Value"]

correlation = st.pearsonr(cosprings_sssm_unemp_x,cosprings_sssm_value_y)
print(f"The correlation between the unemployment rate and median sale price is: {correlation[0]}.")
print(f"P-Value is: {correlation[1]}.")

The correlation between the unemployment rate and median sale price is: 0.5700668182323346.
P-Value is: 8.357608436208383e-06.


### Correlation: Denver - Unemployed, Median Sale Price

In [31]:
sssm_denver_df = master_data_df[(master_data_df["Indicator ID"] =="SSSM") & master_data_df["Region Name"].str.contains("Denver") ]
sssm_denver_df = sssm_denver_df.groupby("Year-Month").first()

In [33]:
denver_sssm_unemp_x = sssm_denver_df["Unemployment Rate"]
denver_sssm_value_y = sssm_denver_df["Value"]

correlation = st.pearsonr(denver_sssm_unemp_x,denver_sssm_value_y)
print(f"The correlation between the unemployment rate and median sale price is: {correlation[0]}.")
print(f"P-Value is: {correlation[1]}.")

The correlation between the unemployment rate and median sale price is: 0.4386727635223719.
P-Value is: 0.00101730212996578.


In [ ]:
# No Data on SSSM for Breckenridge, CO
# sssm_df_breck = sssm_df[sssm_df["Region Name"].str.contains("Breckenridge")]
# sssm_df_breck
# sssm_df["Region Name"].unique()

In [46]:
# sssm_greeley_scatter = sssm_df_greely.plot("Unemployment Rate", "Value", kind="scatter")

# # Find Correlation
# correlation = st.pearsonr(greeley_unem_x,greeley_value_y)
# print(f"The correlation between the unemployment rate and median sale price value in Greeley, CO is: {correlation[0]}.")
# print(f"P-Value is: {correlation[1]}.")

# # FIND LINEAR REGRESSION
# greeley_unem_x = sssm_df_greely["Unemployment Rate"]
# greeley_value_y = sssm_df_greely["Value"]

# # Formula
# (slope, intercept, rvalue, pvalue, stderr) = linregress(greeley_unem_x, greeley_value_y)
# greeley_regress_values = greeley_unem_x * slope + intercept

# # Line Equation
# greeley_line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# # Plot
# plt.scatter(greeley_unem_x,greeley_value_y)
# plt.plot(greeley_unem_x,greeley_regress_values,"r-")

# # Design Plot
# plt.annotate(greeley_line_eq,(6, 280000),fontsize=15,color="red")
# plt.xlabel("Unemployment %")
# plt.ylabel('Median Sale Price')

# # Show
# plt.show()